In [72]:
import os
os.chdir(r"c:\projects\PY_MLE")

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from string import punctuation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [55]:
df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [56]:
df.columns

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')

In [57]:
df['headline'].isnull().values.any()

False

In [58]:
headline = df['headline'].to_list()
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [59]:
headline = [word for word in headline if word != 'Unknown']

In [60]:
len(headline)

1214

In [61]:
def repreprocessing(row_sentence):
    preprocessed_sentence = row_sentence.encode('utf-8').decode('ascii','ignore')
    return ''.join(word for word in preprocessed_sentence if word not in punctuation).lower()

In [62]:
repreprocessing_headline = [repreprocessing(word) for word in headline]
repreprocessing_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [63]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(repreprocessing_headline)
vocab_size = len(tokenizer.word_index) + 1
print("vocab_size:{}".format(vocab_size))

vocab_size:3494


In [64]:
sequences = list()

for sentence in repreprocessing_headline:
    encoded = tokenizer.texts_to_sequences([sentence])[0]

    for i in range(1,len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print(sequences[2])
print(sequences[3])

[99, 269, 371, 1115]
[99, 269, 371, 1115, 582]


In [65]:
tokenizer.word_index.items()

dict_items([('the', 1), ('a', 2), ('to', 3), ('of', 4), ('in', 5), ('for', 6), ('and', 7), ('is', 8), ('on', 9), ('with', 10), ('trump', 11), ('as', 12), ('at', 13), ('new', 14), ('how', 15), ('from', 16), ('it', 17), ('an', 18), ('that', 19), ('be', 20), ('season', 21), ('us', 22), ('you', 23), ('its', 24), ('what', 25), ('episode', 26), ('can', 27), ('your', 28), ('not', 29), ('he', 30), ('now', 31), ('his', 32), ('are', 33), ('teaching', 34), ('war', 35), ('out', 36), ('no', 37), ('was', 38), ('by', 39), ('trumps', 40), ('has', 41), ('over', 42), ('may', 43), ('into', 44), ('why', 45), ('more', 46), ('we', 47), ('who', 48), ('about', 49), ('recap', 50), ('activities', 51), ('1', 52), ('just', 53), ('do', 54), ('women', 55), ('when', 56), ('syria', 57), ('trade', 58), ('i', 59), ('2', 60), ('or', 61), ('will', 62), ('this', 63), ('have', 64), ('president', 65), ('but', 66), ('home', 67), ('up', 68), ('long', 69), ('one', 70), ('off', 71), ('facebook', 72), ('house', 73), ('gop', 74),

In [66]:
max_len = max(len(x) for x in sequences)
max_len

24

In [67]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences[:2]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,  99, 269],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,  99, 269, 371]])

In [68]:
# 라벨 분리
sequences = np.array(sequences)
x = sequences[:, :-1]
y = sequences[:, -1]

In [69]:
print(x[:5])
print(y[:5])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0   99  269  371 1115]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   99  269  371 1115  582]]
[ 269  371 1115  582   52]


In [70]:
# 라벨만 원핫코딩
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [73]:
embedding_dim = 10
hidden_units = 128

model = Sequential(
    [
        Embedding(vocab_size, embedding_dim),
        LSTM(hidden_units),
        Dense(vocab_size, activation='softmax')
    ]
)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x,y,epochs=200,verbose=1)



Epoch 1/200


244/244 [==============================] - 4s 9ms/step - loss: 7.6486 - accuracy: 0.0273
Epoch 2/200
244/244 [==============================] - 2s 9ms/step - loss: 7.1147 - accuracy: 0.0299
Epoch 3/200
244/244 [==============================] - 2s 9ms/step - loss: 6.9734 - accuracy: 0.0341
Epoch 4/200
244/244 [==============================] - 2s 9ms/step - loss: 6.8451 - accuracy: 0.0400
Epoch 5/200
244/244 [==============================] - 2s 9ms/step - loss: 6.6858 - accuracy: 0.0445
Epoch 6/200
244/244 [==============================] - 2s 9ms/step - loss: 6.5052 - accuracy: 0.0506
Epoch 7/200
244/244 [==============================] - 2s 9ms/step - loss: 6.3070 - accuracy: 0.0536
Epoch 8/200
244/244 [==============================] - 2s 9ms/step - loss: 6.0996 - accuracy: 0.0582
Epoch 9/200
244/244 [==============================] - 2s 9ms/step - loss: 5.9001 - accuracy: 0.0618
Epoch 10/200
244/244 [==============================] - 2s 9ms/step - loss: 5.7140 - ac

In [76]:
def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ''

    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre') # max_len 라벨분리하기전이니까 -1

        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            if index == result:
                break

        current_word = current_word + " " + word
        sentence = sentence + " " + word
    sentence = init_word + sentence
    return sentence

In [77]:
# 예측을 해보자
print(sentence_generation(model, tokenizer, 'i', 10))

i disapprove of school vouchers can i still apply for them


In [78]:
print(sentence_generation(model, tokenizer, 'how', 10))

how can a doctor grapple with the epidemic of cost now
